In [30]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('NEU Metal Surface Defects Data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

NEU Metal Surface Defects Data/Thumbs.db
NEU Metal Surface Defects Data/train/Crazing/Cr_122.bmp
NEU Metal Surface Defects Data/train/Crazing/Cr_297.bmp
NEU Metal Surface Defects Data/train/Crazing/Cr_217.bmp
NEU Metal Surface Defects Data/train/Crazing/Cr_249.bmp
NEU Metal Surface Defects Data/train/Crazing/Cr_291.bmp
NEU Metal Surface Defects Data/train/Crazing/Cr_238.bmp
NEU Metal Surface Defects Data/train/Crazing/Cr_37.bmp
NEU Metal Surface Defects Data/train/Crazing/Cr_32.bmp
NEU Metal Surface Defects Data/train/Crazing/Cr_25.bmp
NEU Metal Surface Defects Data/train/Crazing/Cr_42.bmp
NEU Metal Surface Defects Data/train/Crazing/Cr_198.bmp
NEU Metal Surface Defects Data/train/Crazing/Cr_165.bmp
NEU Metal Surface Defects Data/train/Crazing/Cr_270.bmp
NEU Metal Surface Defects Data/train/Crazing/Cr_87.bmp
NEU Metal Surface Defects Data/train/Crazing/Cr_160.bmp
NEU Metal Surface Defects Data/train/Crazing/Cr_213.bmp
NEU Metal Surface Defects Data/train/Crazing/Cr_274.bmp
NEU Metal Su

In [2]:
train_dir = "./NEU Metal Surface Defects Data/train"
test_dir = "./NEU Metal Surface Defects Data/test"
valid_dir = "./NEU Metal Surface Defects Data/valid"

# Data preprocessing before modeling

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                             rotation_range=20,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)


#get the images from train datagen
train_generator = train_datagen.flow_from_directory(train_dir,
                                              target_size=(200, 200),
                                              batch_size=32,
                                              class_mode='categorical',
                                              shuffle=True)
valid_generator = test_datagen.flow_from_directory(valid_dir,
                                              target_size=(200, 200),
                                              batch_size=32,
                                              class_mode='categorical',
                                              shuffle=False)

test_generator = test_datagen.flow_from_directory(test_dir,
                                              target_size=(200, 200),
                                              batch_size=32,
                                              class_mode='categorical',
                                              shuffle=False)

In [ ]:
#checking for batch size
for image_batch , labels_batch in train_generator :
    print(image_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
#checking for class names
class_names = train_generator.class_indices
class_names = list(class_names.keys())
print(class_names)

# Sample images 

In [6]:
#making functions
def get_sample_image(generator):
    images, labels = next(generator)
    image = images[0]
    label_index = np.argmax(labels[0])
    label_name = class_names[label_index]

    return image, label_name


def sample_images(generator, nrows=3, ncols=3):
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10, 10))
    
    for i in range(nrows*ncols):
        image, label_name = get_sample_image(generator)
        row = i // ncols
        col = i % ncols
        ax = axes[row][col]
        ax.imshow(image)
        ax.set_title(label_name)
        ax.axis('off')

    plt.show()

In [ ]:
sample_images(train_generator, nrows=3, ncols=3)

In [ ]:
image_batch

# Deep learning model using CNN

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout
from tensorflow.keras.applications.efficientnet import EfficientNetB7

In [ ]:
model1 = Sequential([ Conv2D(32, (2, 2), activation='relu', input_shape=(200, 200, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (2, 2), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (2, 2), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(6 ,activation='softmax')])

In [11]:
model1.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
model1.summary()

In [ ]:
history = model1.fit(train_generator,
                    epochs=20,
                    batch_size=32,
                    validation_data=valid_generator)

# Model 1 Evaluate

In [14]:
def plot_history(history,metric):
    plt.plot(history.history[metric])
    plt.plot(history.history["val_"+metric],"")
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, "val_"+metric])
    plt.show()

In [ ]:
plot_history(history,"accuracy")

In [ ]:
plot_history(history,"loss")

In [ ]:
result = model1.evaluate(test_generator)
print("Test loss, Test accuracy : ", result)

In [ ]:
base_model = tf.keras.applications.efficientnet.EfficientNetB7(include_top = False , weights = 'imagenet' ,
                                                               input_shape = (200,200,3), pooling= 'max')
model2 = Sequential([ Conv2D(32, (2, 2), activation='relu', input_shape=(200, 200, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (2, 2), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (2, 2), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(6 ,activation='softmax')])

In [19]:
model2.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
model2.summary()

In [ ]:
history2 = model2.fit(train_generator,
                    epochs=20,
                    batch_size=32,
                    validation_data=valid_generator)

# Model 2 Evaluate

In [22]:
def plot_history(history, metric):
    plt.plot(history2.history[metric])
    plt.plot(history2.history['val_'+metric], '')
    plt.xlabel('Epochs')
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])
    plt.show()

In [ ]:
plot_history(history, 'accuracy')

In [ ]:
plot_history(history, 'loss')

In [ ]:
result = model2.evaluate(test_generator)
print("Test loss, Test accuracy : ", result)

# Visualize Predicted Images

In [ ]:
images, labels = next(test_generator)    

indices = np.random.choice(range(len(images)), size=9)
images = images[indices]
labels = labels[indices]

predictions = model1.predict(images)


class_names=list(test_generator.class_indices.keys())


plt.figure(figsize=(10,10))
    
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
        
    image = images[i]
        
    if image.shape[-1] == 1:
        image = np.squeeze(image) 
        
    plt.imshow(image)
        
    predicted_label = np.argmax(predictions[i])
        
    if predicted_label == np.argmax(labels[i]):
        color='blue'
        result_text="Correct"
            
    else:
        color='red'
        result_text="Incorrect"

    label_text="True: "+ class_names[np.argmax(labels[i])] + ", Pred: " + class_names[predicted_label] + f" ({result_text})"        
            
    plt.xlabel(label_text,color=color)

In [ ]:
images, labels = next(test_generator)    

indices = np.random.choice(range(len(images)), size=9)
images = images[indices]
labels = labels[indices]

predictions = model2.predict(images)


class_names=list(test_generator.class_indices.keys())


plt.figure(figsize=(10,10))
    
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
        
    image = images[i]
        
    if image.shape[-1] == 1:
        image = np.squeeze(image) 
        
    plt.imshow(image)
        
    predicted_label = np.argmax(predictions[i])
        
    if predicted_label == np.argmax(labels[i]):
        color='blue'
        result_text="Correct"
            
    else:
        color='red'
        result_text="Incorrect"

    label_text="True: "+ class_names[np.argmax(labels[i])] + ", Pred: " + class_names[predicted_label] + f" ({result_text})"        
            
    plt.xlabel(label_text,color=color)

# Comparing between two models

In [ ]:
import pandas as pd

# Define the accuracy and loss values for both models
model1_accuracy = history.history['accuracy'][-1]
model1_loss = history.history['loss'][-1]
model2_accuracy = history2.history['accuracy'][-1]
model2_loss = history2.history['loss'][-1]


# Create a dictionary with the accuracy and loss values
data = {'Model': ['Model 1', 'Model 2'],
        'Accuracy': [model1_accuracy, model2_accuracy],
        'Loss': [model1_loss, model2_loss]}

df = pd.DataFrame(data)
df

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

# 1) Modify your code to extract features from your trained model
#    We can define a "feature-extractor" model that stops at the second-to-last layer

feature_model = keras.Model(
    inputs=model1.input,
    outputs=model1.layers[-2].output  # e.g., the Dense(256) layer, if your final layer is Dense(6)
)

# 2) Collect features for all training images
train_features = []
train_labels = []  # optional, just to compare if you'd like

train_generator.reset()
for batch_index in range(len(train_generator)):
    images, labels = next(train_generator, (None, None))
    if images is None:
        break
    feats = feature_model.predict(images)
    train_features.append(feats)
    train_labels.append(labels)

train_features = np.concatenate(train_features, axis=0)
train_labels = np.concatenate(train_labels, axis=0)

print("Shape of train_features:", train_features.shape)

# 3) Run k-means (assuming 6 known defect classes):
kmeans = KMeans(n_clusters=6, random_state=42)
kmeans.fit(train_features)

# 4) Inference on new data:
test_generator.reset()
for batch_index in range(len(test_generator)):
    images, labels = next(test_generator, (None, None))
    if images is None:
        break

    # a) CNN’s softmax classification
    cnn_preds = model1.predict(images)
    top_class = np.argmax(cnn_preds, axis=1)

    # b) Feature-based clustering
    feats_test = feature_model.predict(images)
    cluster_assignments = kmeans.predict(feats_test)

    # c) Distance from cluster center 
    distances = []
    for i in range(len(cluster_assignments)):
        center = kmeans.cluster_centers_[cluster_assignments[i]]
        dist = np.linalg.norm(feats_test[i] - center)
        distances.append(dist)

    # Threshold for novelty
    DIST_THRESHOLD = 10.0  # You’ll pick or tune

    for i, dist in enumerate(distances):
        if dist > DIST_THRESHOLD:
            print(f"Image {i} might be novel! Distance to cluster center: {dist:.2f}")
            # Prompt user or handle it as you like
